In [2]:
import tensorflow as tf

In [20]:
import pathlib
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
from functools import reduce
from collections import defaultdict

import pandas as pd
import geopandas as gpd
import folium
import shapely
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import xgboost
import sklearn.cluster

In [6]:
input_path = 'E:\yujin\INHA\SBJ_2007_001'
p = pathlib.Path(input_path)

In [7]:
자전거스테이션 = pd.read_csv(p.joinpath('02.자전거스테이션.csv'))
자전거스테이션

,Station_ID,STATION_NAME,거치대 수량,위도,경도
0,101,어울림마을 701동 앞,20,37.654775,126.834584
1,103,대림e-편한세상106동,20,37.660442,126.840377
2,104,탄현마을8단지,25,37.698523,126.766042
3,105,KT 덕양지사 앞,20,37.655244,126.839261
4,106,원당역 앞 공영주차장,30,37.653410,126.842530
...,...,...,...,...,...
159,350,★하이파크5단지 502동앞 버스정류장,20,37.697867,126.753089
160,351,◆일산에듀포레 푸르지오,40,37.702259,126.767231
161,352,◆꿈에그린203동앞,40,37.666425,126.749244
162,353,◆꿈에그린106동앞,40,37.666720,126.750784


In [8]:
자전거스테이션.describe(include='all')

,Station_ID,STATION_NAME,거치대 수량,위도,경도
count,164.000000,164,164.000000,164.000000,164.000000
unique,NaN,164,NaN,NaN,NaN
top,NaN,달빛마을3단지 상가 앞,NaN,NaN,NaN
freq,NaN,1,NaN,NaN,NaN
mean,231.987805,NaN,26.189024,37.658544,126.799452
std,96.579919,NaN,7.087161,0.024517,0.043810
min,101.000000,NaN,4.000000,37.591856,126.717535
25%,161.750000,NaN,20.000000,37.643943,126.767998
50%,228.500000,NaN,25.000000,37.660869,126.785039
75%,306.250000,NaN,30.000000,37.675909,126.832985


In [9]:
for column in ['Station_ID']:
    자전거스테이션[column] = 자전거스테이션[column].apply(str)
    
자전거스테이션.describe(include='all')

,Station_ID,STATION_NAME,거치대 수량,위도,경도
count,164,164,164.000000,164.000000,164.000000
unique,164,164,NaN,NaN,NaN
top,302,달빛마을3단지 상가 앞,NaN,NaN,NaN
freq,1,1,NaN,NaN,NaN
mean,NaN,NaN,26.189024,37.658544,126.799452
std,NaN,NaN,7.087161,0.024517,0.043810
min,NaN,NaN,4.000000,37.591856,126.717535
25%,NaN,NaN,20.000000,37.643943,126.767998
50%,NaN,NaN,25.000000,37.660869,126.785039
75%,NaN,NaN,30.000000,37.675909,126.832985


In [10]:
인구분포도 = gpd.read_file(p.joinpath('06.인구(거주)분포도(100M_X_100M).geojson'))
인구분포도

,gid,val,geometry
0,다사268650,NaN,"MULTIPOLYGON (((126.66985 37.68162, 126.66984 ..."
1,다사268651,NaN,"MULTIPOLYGON (((126.66984 37.68252, 126.66983 ..."
2,다사268652,NaN,"MULTIPOLYGON (((126.66983 37.68342, 126.66982 ..."
3,다사268653,NaN,"MULTIPOLYGON (((126.66982 37.68432, 126.66981 ..."
4,다사268654,NaN,"MULTIPOLYGON (((126.66981 37.68522, 126.66980 ..."
...,...,...,...
27345,다사553641,NaN,"MULTIPOLYGON (((126.99310 37.67534, 126.99310 ..."
27346,다사553642,NaN,"MULTIPOLYGON (((126.99310 37.67624, 126.99309 ..."
27347,다사553643,NaN,"MULTIPOLYGON (((126.99309 37.67714, 126.99308 ..."
27348,다사553644,NaN,"MULTIPOLYGON (((126.99308 37.67804, 126.99308 ..."


In [11]:
인구분포도.drop('geometry', axis=1).describe(include='all')

,gid,val
count,27350,9241.000000
unique,27350,NaN
top,다사465556,NaN
freq,1,NaN
mean,NaN,114.496483
std,NaN,208.173625
min,NaN,0.000000
25%,NaN,0.000000
50%,NaN,10.000000
75%,NaN,144.000000


In [12]:
인구분포도.loc[인구분포도['val'].apply(pd.isna), 'val'] = 0
인구분포도

,gid,val,geometry
0,다사268650,0.0,"MULTIPOLYGON (((126.66985 37.68162, 126.66984 ..."
1,다사268651,0.0,"MULTIPOLYGON (((126.66984 37.68252, 126.66983 ..."
2,다사268652,0.0,"MULTIPOLYGON (((126.66983 37.68342, 126.66982 ..."
3,다사268653,0.0,"MULTIPOLYGON (((126.66982 37.68432, 126.66981 ..."
4,다사268654,0.0,"MULTIPOLYGON (((126.66981 37.68522, 126.66980 ..."
...,...,...,...
27345,다사553641,0.0,"MULTIPOLYGON (((126.99310 37.67534, 126.99310 ..."
27346,다사553642,0.0,"MULTIPOLYGON (((126.99310 37.67624, 126.99309 ..."
27347,다사553643,0.0,"MULTIPOLYGON (((126.99309 37.67714, 126.99308 ..."
27348,다사553644,0.0,"MULTIPOLYGON (((126.99308 37.67804, 126.99308 ..."


In [13]:
전철역 = pd.read_csv(p.joinpath('19.전철역_공간정보.csv'))
전철역

,rail_nm,station_nm,lot_num_addr,road_nm_addr,X,Y
0,3호선,대곡,경기도 고양시 덕양구 대장동 420-1,경기도 고양시 덕양구 대주로107번길 71-81,126.809614,37.632306
1,3호선,대화,경기도 고양시 일산서구 대화동 2221,경기도 고양시 일산서구 중앙로 지하 1569,126.747206,37.675846
2,경의중앙,일산,경기도 고양시 일산서구 일산동 655-691,경기도 고양시 일산서구 경의로 672,126.770067,37.682007
3,3호선,백석,경기도 고양시 일산동구 백석동 1538,경기도 고양시 일산동구 중앙로 지하 1042,126.788148,37.642953
4,경의중앙,대곡,경기도 고양시 덕양구 대장동 420-1,경기도 고양시 덕양구 대주로 107번길 71-81,126.811047,37.631615
5,경의중앙,강매,경기도 고양시 덕양구 행신동 1115-1,경기도 고양시 덕양구 소원로 202,126.844436,37.612290
6,경의중앙,백마,경기도 고양시 일산동구 백석동 92,경기도 고양시 일산동구 경의로 318,126.794067,37.658431
7,경의중앙,능곡,경기도 고양시 덕양구 토당동 454-3,경기도 고양시 덕양구 토당로 35,126.820838,37.618765
8,경의중앙,풍산,경기도 고양시 일산동구 풍동 1042,경기도 고양시 일산동구 경의로 486,126.786528,37.671815
9,경의중앙,행신,경기도 고양시 덕양구 행신동 812,경기도 고양시 덕양구 소원로 114,126.834117,37.612170


In [14]:
버스정류소 = pd.read_csv(p.joinpath('20.고양시_버스정류소.csv'))
버스정류소

,STATION_NM,STATION_ID,X,Y
0,신일비즈니스고등학교,219000619,126.760592,37.683296
1,중앙공원삼거리,219000587,126.776989,37.688934
2,문촌마을14.19단지,219000624,126.752451,37.671011
3,문촌마을14.19단지,219000623,126.752452,37.671448
4,중앙공원삼거리,219000586,126.777412,37.688771
...,...,...,...,...
2227,고양시청후문,218000940,126.832676,37.659090
2228,사리현동문아파트,219000927,126.845347,37.699382
2229,서울시립고양병원입구,218000530,126.852005,37.712236
2230,옥빛마을14.15단지,218001000,126.833325,37.628699


In [15]:
도로명주소_건물 = gpd.read_file(p.joinpath('15.도로명주소_건물.geojson'))
도로명주소_건물

,BDTYP_CD,BULD_NM,BULD_NM_DC,EMD_CD,GRO_FLO_CO,SIG_CD,UND_FLO_CO,geometry
0,01003,None,None,105,4,41281,0,"MULTIPOLYGON (((126.87256 37.62531, 126.87259 ..."
1,02003,헤리티지,3단지,113,4,41285,1,"MULTIPOLYGON (((126.77481 37.70779, 126.77485 ..."
2,02003,헤리티지,2단지,113,4,41285,1,"MULTIPOLYGON (((126.77510 37.70767, 126.77515 ..."
3,03999,None,None,111,3,41281,0,"MULTIPOLYGON (((126.89398 37.65698, 126.89411 ..."
4,03005,None,None,103,5,41281,0,"MULTIPOLYGON (((126.89124 37.66385, 126.89140 ..."
...,...,...,...,...,...,...,...,...
79997,03012,None,None,132,1,41281,0,"MULTIPOLYGON (((126.88868 37.59669, 126.88878 ..."
79998,21999,None,None,132,1,41281,0,"MULTIPOLYGON (((126.88879 37.59669, 126.88889 ..."
79999,03001,None,None,115,1,41281,0,"MULTIPOLYGON (((126.90831 37.69544, 126.90847 ..."
80000,01003,None,None,132,4,41281,0,"MULTIPOLYGON (((126.89266 37.60498, 126.89280 ..."


In [16]:
for column in ['GRO_FLO_CO', 'UND_FLO_CO']:
    도로명주소_건물[column] = 도로명주소_건물[column].apply(str)
    
도로명주소_건물.drop('geometry', axis=1).describe(include='all')

,BDTYP_CD,BULD_NM,BULD_NM_DC,EMD_CD,GRO_FLO_CO,SIG_CD,UND_FLO_CO
count,80002,9008,10310,80002,80002,80002,80002
unique,220,3348,1049,32,48,3,8
top,01001,후곡마을,가동,104,1,41281,0
freq,28043,219,673,7783,59311,36671,77384


In [17]:
# 5분동안 걷는 최고걸이
도보속도 = 133.1
거리 = 5 * 133.1 / 100 * 60
거리

399.3

In [18]:
축척 = sum([
    *(인구분포도['geometry'].bounds['maxx'] - 인구분포도['geometry'].bounds['minx']),
    *(인구분포도['geometry'].bounds['maxy'] - 인구분포도['geometry'].bounds['miny']),
]) / (2 * len(인구분포도)) / 100
축척

1.0247175113158795e-05

# 학습 데이터 생성
#### 자전거 스테이션별 인구분포 평균, 버스정류장 존재 여부, 건물용도별 존재 여부

In [23]:
def make_데이터(경도, 위도):
    batch_size = 32
    BDTYP_CD = 도로명주소_건물['BDTYP_CD'].unique().tolist()
    BDTYP_CD.sort()
    
    
    #geojson파일의 geometry열은 centroid.x랑 centroid.y 해줘야함
    인구분포도_경위도 = np.stack([인구분포도['geometry'].centroid.x, 인구분포도['geometry'].centroid.y], axis=1).reshape([1, -1, 2]) #(27350,2) --> (1, 27350, 2)
    전철역_경위도 = np.stack([전철역['X'], 전철역['Y']], axis=1)
    버스정류소_경위도 = np.stack([버스정류소['X'], 버스정류소['Y']], axis=1)
    도로명주소_건물_경위도 = np.stack([도로명주소_건물['geometry'].centroid.x, 도로명주소_건물['geometry'].centroid.y], axis=1).reshape([1, -1, 2])
    도로명주소_건물_속성 = 도로명주소_건물.loc[:, 'BDTYP_CD'].apply(BDTYP_CD.index).values.reshape([1, -1])
    
    # np.ceil : 각 원소 값보다 크거나 같은 가장 작은 정수 값
    데이터 = pd.DataFrame()
    for i in tqdm(range(int(np.ceil(len(경도) / batch_size)))): #range(0,6)
        s, e = batch_size * i, batch_size * (i+1)
        자전거스테이션_경위도 = np.stack([경도[s:e], 위도[s:e]], axis=1).reshape([-1, 1, 2])
        인구분포_조건 = np.sum((자전거스테이션_경위도 - 인구분포도_경위도)**2, axis=2)**0.5 < 거리 * 축척  #shape(32,27350)
        인구분포 = np.array([인구분포도.loc[x, 'val'].mean() for x in 인구분포_조건])   #shape(32,)
        전철역_조건 = np.any(np.sum((자전거스테이션_경위도 - 전철역_경위도)**2, axis=2)**0.5 < 거리 * 축척, axis=1)   #shape(32,)
        버스정류소_조건 = np.any(np.sum((자전거스테이션_경위도 - 버스정류소_경위도)**2, axis=2)**0.5 < 거리 * 축척, axis=1)   #shape(32,)
        도로명주소_건물_조건 = np.sum((자전거스테이션_경위도 - 도로명주소_건물_경위도)**2, axis=2)**0.5 < 거리 * 축척    #shape(32,80002)
        도로명주소_건물_데이터 = np.zeros([len(BDTYP_CD), len(자전거스테이션_경위도)])    #shape(220,32)
        for i, x in enumerate(도로명주소_건물_속성 * 도로명주소_건물_조건):  #shape(32,80002)
            x = set(x)
            도로명주소_건물_데이터[:, i] = pd.Series(range(len(BDTYP_CD))).apply(x.__contains__).values #__contains__ : 아이템 존재하면 true
            #도로명주소_건물_데이터 : (220,32)의 True/False dataframe
            
        데이터 = 데이터.append(pd.DataFrame(
            columns=['인구분포', '전철역', '버스정류소', *BDTYP_CD],
            data=np.stack([인구분포, 전철역_조건, 버스정류소_조건, *도로명주소_건물_데이터], axis=1),
        ), ignore_index=True)
    return 데이터

학습데이터 = make_데이터(자전거스테이션['경도'], 자전거스테이션['위도'])
학습데이터.index = 자전거스테이션['Station_ID']
학습데이터

,인구분포,전철역,버스정류소,01001,01002,01003,01004,02001,02002,02003,...,19007,19999,20002,20003,20999,21006,21999,27999,90001,90002
Station_ID,,,,,,,,,,,,,,,,,,,,,
101,240.220000,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103,311.423077,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
104,332.300000,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
105,271.884615,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
106,116.222222,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,239.431373,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
351,290.730769,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
352,186.980769,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [21]:
인구분포도_경위도 = np.stack([인구분포도['geometry'].centroid.x, 인구분포도['geometry'].centroid.y], axis=1).reshape([1, -1, 2]) #(27350,2) --> (1, 27350, 2)

In [22]:
인구분포도_경위도

array([[[126.67040725,  37.68207278],
        [126.67039721,  37.68297402],
        [126.67038717,  37.68387527],
        ...,
        [126.99365281,  37.67759437],
        [126.99364668,  37.67849568],
        [126.9948418 ,  37.66948749]]])

In [164]:
len(BDTYP_CD)

220

In [67]:
range(int(np.ceil(164 / 32)))

range(0, 6)

In [69]:
for i in tqdm(range(int(np.ceil(164 / 32)))):
    print(i)

0
1
2
3
4
5



In [105]:
자전거스테이션_경위도 = np.stack([자전거스테이션['경도'][0:32], 자전거스테이션['위도'][0:32]], axis=1).reshape([-1, 1, 2])

In [117]:
자전거스테이션_경위도.shape

(32, 1, 2)

In [106]:
인구분포_조건 = np.sum((자전거스테이션_경위도 - 인구분포도_경위도)**2, axis=2)**0.5 < 거리 * 축척

In [110]:
인구분포_조건.shape

(32, 27350)

In [157]:
인구분포.shape

(32,)

In [126]:
전철역_경위도 = np.stack([전철역['X'], 전철역['Y']], axis=1)

In [114]:
전철역_경위도.shape

(21, 2)

In [119]:
전철역_조건 = np.any(np.sum((자전거스테이션_경위도 - 전철역_경위도)**2, axis=2)**0.5 < 거리 * 축척, axis=1)

In [122]:
(자전거스테이션_경위도 - 전철역_경위도).shape

(32, 21, 2)

In [128]:
버스정류소_경위도 = np.stack([버스정류소['X'], 버스정류소['Y']], axis=1)
버스정류소_조건 = np.any(np.sum((자전거스테이션_경위도 - 버스정류소_경위도)**2, axis=2)**0.5 < 거리 * 축척, axis=1)

In [130]:
버스정류소_조건.shape

(32,)

In [131]:
도로명주소_건물_경위도 = np.stack([도로명주소_건물['geometry'].centroid.x, 도로명주소_건물['geometry'].centroid.y], axis=1).reshape([1, -1, 2])
도로명주소_건물_조건 = np.sum((자전거스테이션_경위도 - 도로명주소_건물_경위도)**2, axis=2)**0.5 < 거리 * 축척

<ipython-input-131-10d34db697cb>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  도로명주소_건물_경위도 = np.stack([도로명주소_건물['geometry'].centroid.x, 도로명주소_건물['geometry'].centroid.y], axis=1).reshape([1, -1, 2])


In [132]:
도로명주소_건물_조건.shape

(32, 80002)

In [133]:
도로명주소_건물.shape

(80002, 8)

In [155]:
도로명주소_건물_데이터 = np.zeros([len(BDTYP_CD), len(자전거스테이션_경위도)])
도로명주소_건물_데이터.shape

(220, 32)

In [136]:
len(BDTYP_CD)

220

In [139]:
도로명주소_건물_속성.shape

(1, 80002)

In [140]:
도로명주소_건물_속성

array([[ 2,  6,  6, ..., 11,  2,  0]], dtype=int64)

In [141]:
도로명주소_건물_조건

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [142]:
도로명주소_건물_조건.shape

(32, 80002)

In [148]:
for i, x in enumerate(도로명주소_건물_속성 * 도로명주소_건물_조건):
    print(i,x)

0 [0 0 0 ... 0 0 0]
1 [0 0 0 ... 0 0 0]
2 [0 0 0 ... 0 0 0]
3 [0 0 0 ... 0 0 0]
4 [0 0 0 ... 0 0 0]
5 [0 0 0 ... 0 0 0]
6 [0 0 0 ... 0 0 0]
7 [0 0 0 ... 0 0 0]
8 [0 0 0 ... 0 0 0]
9 [0 0 0 ... 0 0 0]
10 [0 0 0 ... 0 0 0]
11 [0 0 0 ... 0 0 0]
12 [0 0 0 ... 0 0 0]
13 [0 0 0 ... 0 0 0]
14 [0 0 0 ... 0 0 0]
15 [0 0 0 ... 0 0 0]
16 [0 0 0 ... 0 0 0]
17 [0 0 0 ... 0 0 0]
18 [0 0 0 ... 0 0 0]
19 [0 0 0 ... 0 0 0]
20 [0 0 0 ... 0 0 0]
21 [0 0 0 ... 0 0 0]
22 [0 0 0 ... 0 0 0]
23 [0 0 0 ... 0 0 0]
24 [0 0 0 ... 0 0 0]
25 [0 0 0 ... 0 0 0]
26 [0 0 0 ... 0 0 0]
27 [0 0 0 ... 0 0 0]
28 [0 0 0 ... 0 0 0]
29 [0 0 0 ... 0 0 0]
30 [0 0 0 ... 0 0 0]
31 [0 0 0 ... 0 0 0]


In [152]:
aa = (도로명주소_건물_속성 * 도로명주소_건물_조건)[0]

In [150]:
pd.Series(range(len(BDTYP_CD)))

0        0
1        1
2        2
3        3
4        4
      ... 
215    215
216    216
217    217
218    218
219    219
Length: 220, dtype: int64

In [154]:
pd.Series(range(len(BDTYP_CD))).apply(aa.__contains__)

0       True
1       True
2       True
3      False
4       True
       ...  
215    False
216    False
217    False
218    False
219    False
Length: 220, dtype: bool

In [160]:
(np.stack([인구분포도['geometry'].centroid.x, 인구분포도['geometry'].centroid.y], axis=1)).shape

<ipython-input-160-9c5155b52c08>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  (np.stack([인구분포도['geometry'].centroid.x, 인구분포도['geometry'].centroid.y], axis=1)).shape


(27350, 2)

In [161]:
인구분포도['geometry'].centroid.x

<ipython-input-161-efb14f3564d6>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  인구분포도['geometry'].centroid.x


0        126.670407
1        126.670397
2        126.670387
3        126.670377
4        126.670367
            ...    
27345    126.993665
27346    126.993659
27347    126.993653
27348    126.993647
27349    126.994842
Length: 27350, dtype: float64

In [162]:
자전거스테이션['Station_ID']

0      101
1      103
2      104
3      105
4      106
      ... 
159    350
160    351
161    352
162    353
163    992
Name: Station_ID, Length: 164, dtype: object

In [165]:
학습데이터.shape

(164, 223)

In [168]:
자전거스테이션['경도'].shape

(164,)